In [26]:
import pandas as pd
from pathlib import Path

ruta_procesados = Path('../data/processed/')
archivo_andres = ruta_procesados / '_Normalizacion_Base_Siniestros_.xlsx'
archivo_camiilo = ruta_procesados / 'base_anuario_de_siniestralidad_normalizado.xlsx'


In [27]:
###
# 1. Verificar si los archivos existen
###
print("Verificando archivos...")
print(f"Archivo de Andrés existe: {archivo_andres.exists()}")
print(f"Tu archivo existe: {archivo_camiilo.exists()}")

Verificando archivos...
Archivo de Andrés existe: True
Tu archivo existe: True


In [28]:
###
# 2. Cargar y consolidar los archivos (Función)
###
def consolidar_excels(archivo1, archivo2, archivo_salida='../data/processed/base_anuario_de_siniestralidad_consolidado_final.xlsx'):
    """
    Consolida dos archivos de Excel, poniendo primero todas las hojas del primero
    y luego todas las hojas del segundo
    """
    
    # Diccionario para guardar todos los DataFrames
    todos_dataframes = {}
    
    # Cargar hojas del archivo de Andrés
    print("\n  Cargando hojas de Andrés...")    
    try:
        andres_sheets = pd.read_excel(archivo1, sheet_name=None)
        for sheet_name, df in andres_sheets.items():
            todos_dataframes[sheet_name] = df
            print(f"  Cargada: {sheet_name}")
    except Exception as e:
        print(f"  Error cargando archivo de Andrés: {e}")
        return
    
    # Cargar hojas del archivo de Camiilo
    print("\n  Cargando tus hojas...")
    try:
        tus_sheets = pd.read_excel(archivo2, sheet_name=None)
        for sheet_name, df in tus_sheets.items():
            todos_dataframes[sheet_name] = df
            print(f"  Cargada: {sheet_name}")
    except Exception as e:
        print(f"  Error cargando tu archivo: {e}")
        return
    
    # Guardar el archivo consolidado
    print(f"\n  Guardando archivo consolidado...")
    try:
        with pd.ExcelWriter(archivo_salida, engine='openpyxl') as writer:
            for sheet_name, df in todos_dataframes.items():
                df.to_excel(writer, sheet_name=sheet_name, index=False)
                print(f"  Guardada hoja: {sheet_name}")
        
        print(f"\n  ¡Consolidación completada!")
        print(f"  Total de hojas: {len(todos_dataframes)}")
        print(f"  Archivo guardado como: {archivo_salida}")
        
    except Exception as e:
        print(f"  Error guardando archivo consolidado: {e}")

In [29]:
###
# 2. Cargar y consolidar los archivos (Función)
###
print("🚀 Iniciando consolidación de archivos...")
consolidar_excels(archivo_andres, archivo_camiilo)

🚀 Iniciando consolidación de archivos...

  Cargando hojas de Andrés...
  Cargada: Factores_Riesgo
  Cargada: Tipo_Transporte
  Cargada: _siniestros_Normalizada

  Cargando tus hojas...
  Cargada: Vehiculos
  Cargada: Actor_Vial

  Guardando archivo consolidado...
  Guardada hoja: Factores_Riesgo
  Guardada hoja: Tipo_Transporte
  Guardada hoja: _siniestros_Normalizada
  Guardada hoja: Vehiculos
  Guardada hoja: Actor_Vial

  ¡Consolidación completada!
  Total de hojas: 5
  Archivo guardado como: ../data/processed/base_anuario_de_siniestralidad_consolidado_final.xlsx


In [30]:
###
# 3. Ver todas las hojas
###
excel_path = '../data/processed/base_anuario_de_siniestralidad_consolidado_final.xlsx'

print('Hojas disponibles en el archivo:')
hojas = pd.ExcelFile(excel_path).sheet_names
for i, hoja in enumerate(hojas):
    print(f"   {i+1}. {hoja}")

Hojas disponibles en el archivo:
   1. Factores_Riesgo
   2. Tipo_Transporte
   3. _siniestros_Normalizada
   4. Vehiculos
   5. Actor_Vial


In [31]:
###
# 4. Ver columnas de interes de las hojas principales
###
excel_path = '../data/processed/base_anuario_de_siniestralidad_consolidado_final.xlsx'

hojas_interes = ['_siniestros_Normalizada', 'Vehiculos', 'Actor_Vial']

for hoja in hojas_interes:
    df = pd.read_excel(excel_path, sheet_name=hoja, nrows=0)  # Solo lee encabezados
    print(f"\n{hoja}: {list(df.columns)}")


_siniestros_Normalizada: ['Codigo_Accidente', 'Longitud', 'Latitud', 'Direccion', 'Fecha_Accidente', 'Año_Accidente', 'Mes_Accidente', 'Día_Accidente', 'Día_Semana_Accidente', 'Hora_Accidente', 'Minuto_Accidente', 'Localidad', 'Clase_Accidente', 'Elemento_Choque', 'Tipo_Objeto_Fijo', 'Gravedad_Indicador_Tradicional', 'Gravedad_indicador_30d', 'direccion_normalizada']

Vehiculos: ['Codigo_Accidente', 'Clase', 'Servicio', 'Modalidad', 'Vehiculo_Viajaba_Clasificado', 'Tipo_SITP']

Actor_Vial: ['Codigo_Accidentado', 'Codigo_Accidente', 'Edad', 'Sexo', 'Gravedad_Indicador_Tradicional', 'Muerte_Posterior', 'Gravedad_Indicador_30d', 'Condicion', 'Condicion_Especifica', 'Validación', 'Rango_Edades']


In [32]:
###
# 5. filtrado de Vehiculos y Actor_vial
###
# Leer las hojas de interes
df_siniestros = pd.read_excel(excel_path, sheet_name='_siniestros_Normalizada')
df_vehiculos = pd.read_excel(excel_path, sheet_name='Vehiculos')
df_actor_vial = pd.read_excel(excel_path, sheet_name='Actor_Vial')

# Obtener los codigos de accidente unicos de la hoja siniestros
codigos_validos = df_siniestros['Codigo_Accidente'].unique()

print('Codigos unicos en _siniestros_Normalizada:', len(codigos_validos))
print('Registros en Vehiculos antes del filtro:', len(df_vehiculos))
print('Registros en Actor_Vial antes del filtro:', len(df_actor_vial))

# Aplicar filtro a Vehiculos
df_vehiculos_filtrado = df_vehiculos[df_vehiculos['Codigo_Accidente'].isin(codigos_validos)]

# Aplicar filtro a Actor_Vial
df_actor_vial_filtrado = df_actor_vial[df_actor_vial['Codigo_Accidente'].isin(codigos_validos)]

print('Registros en Vehiculos despues del filtro:', len(df_vehiculos_filtrado))
print('Registros en Actor_Vial despues del filtro:', len(df_actor_vial_filtrado))

print('Filtro completado')

Codigos unicos en _siniestros_Normalizada: 105094
Registros en Vehiculos antes del filtro: 523168
Registros en Actor_Vial antes del filtro: 608155
Registros en Vehiculos despues del filtro: 194602
Registros en Actor_Vial despues del filtro: 233885
Filtro completado


In [33]:
###
# 6. Creación y exportación del documento con las nuevas hojas
###
output_path = '../data/processed/base_anuario_de_siniestralidad_filtrado.xlsx'

# Leer todas las hojas del archivo original
todas_hojas = pd.read_excel(excel_path, sheet_name=None)

# Leer las hojas especificas que vamos a filtrar
df_siniestros = todas_hojas['_siniestros_Normalizada']
df_vehiculos = todas_hojas['Vehiculos']
df_actor_vial = todas_hojas['Actor_Vial']

# Obtener los codigos validos
codigos_validos = df_siniestros['Codigo_Accidente'].unique()

print('Procesando filtros...')
print('Registros en Vehiculos antes:', len(df_vehiculos))
print('Registros en Actor_Vial antes:', len(df_actor_vial))

# Aplicar filtros
df_vehiculos_filtrado = df_vehiculos[df_vehiculos['Codigo_Accidente'].isin(codigos_validos)]
df_actor_vial_filtrado = df_actor_vial[df_actor_vial['Codigo_Accidente'].isin(codigos_validos)]

print('Registros en Vehiculos despues:', len(df_vehiculos_filtrado))
print('Registros en Actor_Vial despues:', len(df_actor_vial_filtrado))

# Crear nuevo diccionario con todas las hojas, reemplazando las dos filtradas
nuevas_hojas = {}
for nombre_hoja, df in todas_hojas.items():
    if nombre_hoja == 'Vehiculos':
        nuevas_hojas[nombre_hoja] = df_vehiculos_filtrado
    elif nombre_hoja == 'Actor_Vial':
        nuevas_hojas[nombre_hoja] = df_actor_vial_filtrado
    else:
        nuevas_hojas[nombre_hoja] = df  # Mantener las otras hojas sin cambios

# Guardar el nuevo archivo Excel
with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
    for nombre_hoja, df in nuevas_hojas.items():
        df.to_excel(writer, sheet_name=nombre_hoja, index=False)
        print(f'Guardada hoja: {nombre_hoja}')

print('Proceso completado')
print(f'Archivo guardado en: {output_path}')

Procesando filtros...
Registros en Vehiculos antes: 523168
Registros en Actor_Vial antes: 608155
Registros en Vehiculos despues: 194602
Registros en Actor_Vial despues: 233885
Guardada hoja: Factores_Riesgo
Guardada hoja: Tipo_Transporte
Guardada hoja: _siniestros_Normalizada
Guardada hoja: Vehiculos
Guardada hoja: Actor_Vial
Proceso completado
Archivo guardado en: ../data/processed/base_anuario_de_siniestralidad_filtrado.xlsx
